# Semi-Automated Embracing Approach Workflow

...

In [ ]:
from embracing import step1, step2, step3, synset_relations
from ipywidgets import widgets
from itables import init_notebook_mode


init_notebook_mode(all_interactive=True)

### Preliminary Threats Upload

Please upload the preliminary threats list in CSV format (suggested path: `./data/<file>.csv`).
Once the file has been uploaded, you can proceed and run the next cell.

In [2]:
threat_list_path = input("Please specify the path to your threat list (csv)")

min_score, max_score = step1(threat_list_path)
print(f"Similarity score range is between {min_score} and {max_score}")

Similarity score range is between -0.1 and 0.82


### Set Semantic Similarity Score Threshold

Please set your desirable semantic similarity score threshold by running the following cell.

In [3]:
ss_threshold = widgets.FloatSlider(min=min_score, max=max_score, step=0.01, description='threshold:', readout_format='.2f')
ss_threshold

FloatSlider(value=0.0, description='threshold:', max=0.82, min=-0.1, step=0.01)

### Threat Pairs Inspection

Now you are able to further investigate all the threat pairs with a score equal or greater to such a threshold. To display the table, please run the following cell.

In [4]:
embraceable_candidates = step2(f'{threat_list_path.split(".csv")[0]}_semantic_similarity_scores.csv', ss_threshold.value)
print(f'The list of embraceable candidates above the threshold {ss_threshold.value} contains {len(embraceable_candidates)} pairs.')
embraceable_candidates

The list of embraceable candidates above the threshold 0.5600000000000002 contains 77 pairs.


sentence1   
148                                             Malware  \
187               Manipulation of hardware and software   
201               Manipulation of hardware and software   
220               Manipulation of hardware and software   
224               Manipulation of hardware and software   
...                                                 ...   
4023                              Insecure API endpoint   
4090                                  V2X eavesdropping   
4319                  Insufficient Data Breach Response   
4352  Non-transparent Policies, Agreements, Terms an...   
4431                     Sharing of Data with 3rd Party   

                                              sentence2     score  
148                                  Mobile App malware  0.573813  
187                         Manipulation of information  0.655521  
201               Software vulnerabilities exploitation  0.589752  
220   An adversary can execute arbitrary code on the...  0.581725  
224   Attacker can moditv the hardware ov changina o...  0.615692  
...                                                 ...       ...  
4023                             Insecure Data Transfer  0.669211  
4090                                            V2X DoS  0.654126  
4319  Insufficient Data Structure Model to handle us...  0.624751  
4352       Inappropriate Policies, Terms and Conditions  0.666409  
4431           Transfer or Processing through 3rd Party  0.813606  

[77 rows x 3 columns]

Once you identified a row of interest, run the following cell and specify the index of such a row.

In [8]:
index_to_embrace = int(input('Please specify the index of the pair you want to embrace.'))
embraceable_candidates.iloc[embraceable_candidates.index==index_to_embrace]


sentence1   
1845  Unintentional change of data or car components...  \

                                              sentence2     score  
1845  Attacker modifies the path of the sensitive da...  0.589352

### Synset Relations Among Nouns in Target Threat Pairs

Now that you shifted the focus of the analysis on a specific threat pair, run the following cell to obtain automatically identified (if present) synset relations.

In [9]:
s1 = embraceable_candidates.iloc[embraceable_candidates.index==index_to_embrace]['sentence1'].values[0]
s2 = embraceable_candidates.iloc[embraceable_candidates.index==index_to_embrace]['sentence2'].values[0]
is_partof, is_typeof = step3(s1, s2)
print(is_partof, is_typeof)

car.n.02 is a hyponym of vehicle.n.01
False True


To support the most appropriate choice of wording/level of detail, run the last cell for an overview of the synset relations related to the nouns identified in both threat labels.

In [10]:
import pandas

synset_dict1 = synset_relations(embraceable_candidates.iloc[embraceable_candidates.index==index_to_embrace]['sentence1'].values[0])
synset_dict2 = synset_relations(embraceable_candidates.iloc[embraceable_candidates.index==index_to_embrace]['sentence2'].values[0])

focus_df = pandas.concat([pandas.DataFrame.from_dict(synset_dict1['terms']), pandas.DataFrame.from_dict(synset_dict2['terms'])])
focus_df

term   
0         Synset('change.n.01')  \
1          Synset('datum.n.01')   
2            Synset('car.n.01')   
3      Synset('component.n.01')   
4  Synset('configuration.n.01')   
0            Synset('way.n.05')   
1          Synset('datum.n.01')   
2        Synset('vehicle.n.01')   

                                            synonyms   
0  [Synset('change.n.01'), Synset('change.n.02'),...  \
1                             [Synset('datum.n.01')]   
2  [Synset('car.n.01'), Synset('car.n.02'), Synse...   
3  [Synset('component.n.01'), Synset('part.n.01')...   
4  [Synset('configuration.n.01'), Synset('shape.n...   
0  [Synset('way.n.05'), Synset('path.n.02'), Syns...   
1                             [Synset('datum.n.01')]   
2  [Synset('vehicle.n.01'), Synset('vehicle.n.02'...   

                                            meronyms holonyms   
0                                                 []       []  \
1                                                 []       []   
2  [Synset('accelerator.n.01'), Synset('air_bag.n...       []   
3                                                 []       []   
4                                                 []       []   
0                                                 []       []   
1                                                 []       []   
2                       [Synset('splashboard.n.02')]       []   

                   hypernyms [L1]   
0      [Synset('happening.n.01')]  \
1    [Synset('information.n.02')]   
2  [Synset('motor_vehicle.n.01')]   
3           [Synset('part.n.09')]   
4         [Synset('design.n.02')]   
0         [Synset('course.n.04')]   
1    [Synset('information.n.02')]   
2     [Synset('conveyance.n.03')]   

                                      hypernyms [L2]   
0   [Synset('happening.n.01'), Synset('event.n.01')]  \
1  [Synset('information.n.02'), Synset('cognition...   
2  [Synset('motor_vehicle.n.01'), Synset('self-pr...   
3      [Synset('part.n.09'), Synset('concept.n.01')]   
4  [Synset('design.n.02'), Synset('arrangement.n....   
0     [Synset('course.n.04'), Synset('action.n.01')]   
1  [Synset('information.n.02'), Synset('cognition...   
2  [Synset('conveyance.n.03'), Synset('instrument...   

                                      hypernyms [L3]   
0  [Synset('happening.n.01'), Synset('event.n.01'...  \
1  [Synset('information.n.02'), Synset('cognition...   
2  [Synset('motor_vehicle.n.01'), Synset('self-pr...   
3  [Synset('part.n.09'), Synset('concept.n.01'), ...   
4  [Synset('design.n.02'), Synset('arrangement.n....   
0  [Synset('course.n.04'), Synset('action.n.01'),...   
1  [Synset('information.n.02'), Synset('cognition...   
2  [Synset('conveyance.n.03'), Synset('instrument...   

                                       hyponyms [L1]   
0  [Synset('acceleration.n.01'), Synset('avulsion...  \
1  [Synset('reading.n.03'), Synset('statistic.n.0...   
2  [Synset('ambulance.n.01'), Synset('beach_wagon...   
3  [Synset('be-all_and_end-all.n.01'), Synset('pl...   
4  [Synset('redundancy.n.03'), Synset('topology.n...   
0  [Synset('ambages.n.01'), Synset('primrose_path...   
1  [Synset('reading.n.03'), Synset('statistic.n.0...   
2  [Synset('bumper_car.n.01'), Synset('craft.n.02...   

                                       hyponyms [L2]   
0  [Synset('acceleration.n.01'), Synset('avulsion...  \
1  [Synset('reading.n.03'), Synset('statistic.n.0...   
2  [Synset('ambulance.n.01'), Synset('beach_wagon...   
3  [Synset('be-all_and_end-all.n.01'), Synset('pl...   
4  [Synset('redundancy.n.03'), Synset('topology.n...   
0  [Synset('ambages.n.01'), Synset('primrose_path...   
1  [Synset('reading.n.03'), Synset('statistic.n.0...   
2  [Synset('bumper_car.n.01'), Synset('craft.n.02...   

                                       hyponyms [L3]  
0  [Synset('acceleration.n.01'), Synset('avulsion...  
1  [Synset('reading.n.03'), Synset('statistic.n.0...  
2  [Synset('ambulance.n.01'), Synset('beach_wagon...  
3  [Synset('be-all_and_end-all.n.01'), Synset(